# Force Atlas 2
# Skip notebook test


**Author: Hugo Linsenmaier**
    
In this notebook, we will see how large graph visualization can be achieved with cuGraph. 

| Author Credit    |    Date    |  Update          | cuGraph Version |  Test Hardware |
| -----------------|------------|------------------|-----------------|----------------|
| Hugo Linsenmaier | 11/16/2020 | created          | 0.17            | GV100, CUDA 11.0
| Brad Rees        | 01/11/2022 | tested / updated | 22.02 nightly   | RTX A6000 48GB CUDA 11.5
| Ralph Liu        | 06/22/2022 | updated/tested   | 22.08 nightly   | V100, CUDA 11.5
| Don Acosta       | 08/01/2022 | tested / updated | 22.08 nightly   | DGX Tesla A100 CUDA 11.5 
| Don Acosta       | 07/17/2023 | tested / updated | 23.08 nightly   |RTX A6000 48GB CUDA 11.7 

# Introduction:


Force Atlas 2 is a force directed layout algorithm where nodes behave as particules and edges as springs. An iterative process will compute attractive and repulsive forces between these entities to converge in an equilibrium state where the drawing is visually interpretable by the user.


See https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0098679 for more details.


Please refer to the [documentation](https://docs.rapids.ai/api/cugraph/stable/api_docs/api/cugraph.force_atlas2.html)  on how to use the different parameters.


In [ ]:
# Import RAPIDS libraries
import cugraph
import time

In [ ]:
# Define the parameters 
ITERATIONS=500
THETA=1.0
OPTIMIZE=True

In [ ]:
# Import a built-in dataset
from cugraph.experimental.datasets import netscience

# cuGraph

In [ ]:
G = netscience.get_graph(fetch=True)
G.number_of_nodes(), G.number_of_edges()

### Force Atlas 2 call

In [ ]:
start = time.time()
pos_gdf = cugraph.layout.force_atlas2(G,
                                  max_iter=ITERATIONS,
                                  pos_list=None,
                                  outbound_attraction_distribution=True,
                                  lin_log_mode=False,
                                  edge_weight_influence=1.0,
                                  jitter_tolerance=1.0,
                                  barnes_hut_optimize=OPTIMIZE,
                                  barnes_hut_theta=THETA,
                                  scaling_ratio=2.0,
                                  strong_gravity_mode=False,
                                  gravity=1.0,
                                  verbose=False,
                                  callback=None)
elapsed = time.time() - start
print("Cugraph time : " + str(elapsed))
pos_gdf.head(5)

## Visualize the graph

The following section creates a visualization of the network using the locations generated by the Force Atlas algorithms.  However, the following section is dependent on having the **cuxfilter** package installed.  

See the cuxfilter GitHub page for installation: https://github.com/rapidsai/cuxfilter.   

Alternatively, the package comes installed in the RAPIDS development Docker container. 
See:  https://hub.docker.com/r/rapidsai/rapidsai-dev/ 

<hr>

Set up the visualization

In [ ]:
import importlib.util
cux_spec = importlib.util.find_spec("cuxfilter")
if cux_spec is None:
    print("Visualization package is not available.")
else:
    from cuxfilter.charts.datashader.custom_extensions.graph_assets import calc_connected_edges
    # Viz libraries
    import holoviews as hv

    from colorcet import fire
    from datashader.bundling import directly_connect_edges, hammer_bundle

    from holoviews.operation.datashader import datashade, dynspread
    from holoviews.operation import decimate

    from dask.distributed import Client

    # Define the parameters 
    ITERATIONS=500
    THETA=1.0
    OPTIMIZE=True

    # Import a built-in dataset
    from cugraph.experimental.datasets import netscience

    # Setup Viz
    client = Client()
    hv.notebook_extension('bokeh','matplotlib')
    decimate.max_samples=20000
    dynspread.threshold=0.01
    datashade.cmap=fire[40:]
    sz = dict(width=150,height=150)
    %opts RGB [xaxis=None yaxis=None show_grid=False bgcolor="black"]

    edges_gdf = netscience.get_edgelist()

    connected = calc_connected_edges(pos_gdf,
                                    edges_gdf,
                                    node_x="x",
                                    node_y="y",
                                    node_x_dtype="float32",
                                    node_y_dtype="float32",
                                    node_id="vertex",
                                    edge_source="src",
                                    edge_target="dst",
                                    edge_aggregate_col=None,
                                    edge_render_type="direct",
                                    )


### Output

In [ ]:
r_direct = None
if cux_spec is not None:
    %opts RGB [tools=["hover"] width=800 height=800]
    r_direct = hv.Curve(connected, label="Direct")
    
r_direct

Copyright (c) 2020 - 2023, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.